In [6]:
#!/usr/bin/env python3
import argparse
import lzma
import os
import pickle
import sys
from typing import Optional
import urllib.request
import sklearn

import numpy as np
import numpy.typing as npt

parser = argparse.ArgumentParser()
# These arguments will be set appropriately by ReCodEx, even if you change them.
parser.add_argument("--predict", default=None, type=str, help="Path to the dataset to predict")
parser.add_argument("--recodex", default=False, action="store_true", help="Running in ReCodEx")
parser.add_argument("--seed", default=42, type=int, help="Random seed")
# For these and any other arguments you add, ReCodEx will keep your default value.
parser.add_argument("--model_path", default="mnist_competition.model", type=str, help="Model path")
args = parser.parse_args([] if "__file__" not in globals() else None)  

In [2]:
class Dataset:
    """MNIST Dataset.

    The train set contains 60000 images of handwritten digits. The data
    contain 28*28=784 values in the range 0-255, the targets are numbers 0-9.
    """
    def __init__(self,
                 name="mnist.train.npz",
                 data_size=None,
                 url="https://ufal.mff.cuni.cz/~courses/npfl129/2526/datasets/"):
        if not os.path.exists(name):
            print("Downloading dataset {}...".format(name), file=sys.stderr)
            urllib.request.urlretrieve(url + name, filename="{}.tmp".format(name))
            os.rename("{}.tmp".format(name), name)

        # Load the dataset, i.e., `data` and optionally `target`.
        dataset = np.load(name)
        for key, value in dataset.items():
            setattr(self, key, value[:data_size])
        self.data = self.data.reshape([-1, 28*28]).astype(float)

In [ ]:
if args.predict is None:
    # We are training a model.
    np.random.seed(args.seed)
    train = Dataset()
    X = train.data
    y = train.target
    train_data, test_data, train_target, test_target = sklearn.model_selection.train_test_split(
                                            X, y, test_size=100, random_state=args.seed)

    # TODO: Train a model on the given dataset and store it in `model`.
    pipe = sklearn.pipeline.Pipeline([
        ('scaler', sklearn.preprocessing.StandardScaler()),
        ('mlp', sklearn.neural_network.MLPClassifier(early_stopping=True))
    ])

    params = {
        'mlp__activation': ['relu', 'logistic', 'tanh', 'identity'],
        'mlp__solver':['lbfgs','sgd','adam'],
        'mlp__hidden_layer_sizes':[100,200,300,400],
        'mlp__learning_rate':['constant','invscaling', 'adaptive'],
        'mlp__alpha':[1e-4, 1e-3]
    }

    model = sklearn.model_selection.GridSearchCV(estimator=pipe, param_grid=params, cv=5)
    model.fit(train_data, train_target)
    print(model.score(test_data, test_target))
    
    # If you trained one or more MLPs, you can use the following code
    # to compress it significantly (approximately 12 times). The snippet
        # assumes the trained `MLPClassifier` is in the `mlp` variable.
    
    model._optimizer = None
    for i in range(len(model.coefs_)): model.coefs_[i] = model.coefs_[i].astype(np.float16)
    for i in range(len(model.intercepts_)): model.intercepts_[i] = model.intercepts_[i].astype(np.float16)

    # Serialize the model.
    with lzma.open(args.model_path, "wb") as model_file:
        pickle.dump(model, model_file)

In [41]:
if args.predict is not None:
    # Use the model and return test set predictions, either as a Python list or a NumPy array.
    test = Dataset(args.predict)

    with lzma.open(args.model_path, "rb") as model_file:
        model = pickle.load(model_file)

    # TODO: Generate `predictions` with the test set predictions.
    predictions = model.predict(test.data)
    print(model.score(test.data, test.target))

    # return predictions

In [ ]:
if __name__ == "__main__":
    main_args = parser.parse_args([] if "__file__" not in globals() else None)
    main(main_args)

In [ ]:
np.random.seed(args.seed)
train = Dataset()
X = train.data
y = train.target
train_data, test_data, train_target, test_target = sklearn.model_selection.train_test_split(
                                        X, y, test_size=100, random_state=args.seed)

# activation_ = {'identity', 'logistic', 'relu', 'tanh'}
# solver_ = {'lbfgs', 'sgd', 'adam'}
# TODO: Train a model on the given dataset and store it in `model`.
model = sklearn.neural_network.MLPClassifier(activation='identity', solver='sgd')
model.fit(train_data, train_target)
print(model.score(test_data, test_target))